# RESTful API Handlers

Nowadays, because of genesis of Microservice architectures, calling RESTful APIs are inevitable. Although this is not a very complex tasks, sometimes it has specific details to make it work efficiently on production. In this notebook I keep some of these details.

## requirments

In [ ]:
# importing
import requests
import urllib.parse
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.auth import HTTPDigestAuth
from enum import Enum
import logging
import time
import json

## API Handler
Having an API Handler object versus calling the API on fly has some benefits. For example, you can use sessions and keep the connections to the APIs to make it way faster.

In [ ]:
class HttpMethod(Enum):
    GET = 1
    POST = 2
    PUT = 3
    DELETE= 4

class SampleAPIHandler:
    
    _api_base = 'https://reqres.in/api/'
    _version = 'v1' # example, not in use here
    
    def __init__(self,
                 timeout=60,
                 wait=2,
                 api_token=None,
                 username=None,
                 password=None,
                 auth_digest=False,
                 logger=None):
        self._timeout = timeout # timeout in seconds
        self._wait = wait # wait time between each request (based on rate quota)
        self._logger = logger or logging.getLogger(self.__class__.__name__)

        # session
        self._session = requests.Session() 

        # api token
        self._api_token = api_token 
        self._session.headers.update({'X-API-TOKEN': api_token})

        # authentication
        if username and password:
            if auth_digest:
                self._session.auth=HTTPDigestAuth(username, password)
            else: # basic authentication
                self._session.auth=(username, password)

    # read env vars
    def read_environment_variable(self, key, required=False):
        value = os.environ.get(key)
        if value is not None:
            return value
        elif required:
            raise ValueError("Environment variable missing: {}".format(key))
        elif required:
            self._logger.warning("Environment variable missing: {}".format(key))
            return None
    
    @classmethod
    def from_environment_vars():
        api_token = read_environment_variable('Sample_API_Token', required=True)
        timeout = read_environment_variable('Sample_API_Timeout')
        wait = read_environment_variable('Sample_API_Wait')
        return SampleAPIHandler(api_token=api_token,
                                timeout=timeout,
                                wait=wait)


    def call_api(self, url, method=HttpMethod.GET, headers=None, params=None, data=None):
        if not data:
            data = {}
        if not params:
            params = {}
            # e.g.
            # params['api_token'] = self._api_token
        if not headers:
            headers = {}
                
        response = None
        try:
            if method == HttpMethod.GET:
                response = self._session.get(url,
                                             headers=headers,
                                             params=params,
                                             timeout=self._timeout)
            elif method == HttpMethod.POST:
                response = self._session.post(url,
                                              headers=headers,
                                              params=params,
                                              data=data,
                                              timeout=self._timeout)
            # wait time
            if self._wait:
                time.sleep(self._wait)

        except requests.exceptions.Timeout:
            self._logger.error('timeout exception for {}'.format(url))

        except requests.exceptions.TooManyRedirects:
            self._logger.error('too many redirects for {}'.format(url))

        except requests.exceptions.ConnectionError:
            self._logger.error('connection error for {}'.format(url))
        
        except: # otherwise
            raise
        
        return response
    

    # check response functions
    def is_response_json(self, response):
        if 'application/json' in response.headers.get('content-type'):
            return True
        return False

    def is_response_redirected(self, response):
        if response.history:
            for h in response.history:
                if h.status_code == 300:
                    return True
        return False
    
    def check_response_headers(self, response):
        # example, check for quota info
        cur_q = response.headers.get('X-Quota-Current')
        max_q = response.headers.get('X-Quota-Allotted')

    def status_is_okey(self, response):
        #  200 OK
        #  201 Created
        #  202 Accepted
        #  203 Non-Authoritative Information
        #  204 No Content
        #  205 Reset Content
        #  206 Partial Content

        if response.status_code in range(200, 206):
            return True
        return False

    def status_is_redirect(self, response):
        #  300  Multiple Choices
        #  301  Moved Permanently
        #  307 Temporary Redirect
        if response.status_code in {300, 301, 307}:
            return True
        return False

    def check_status(self, response):
        pass
        
    # endpoint functions
    def get_users(self, 
                  page=None,
                  offset=None):
        path = '/users'
        url = urllib.parse.urljoin(self._api_base, path)
        headers = {}
        # add pagination info
        if page:
            headers['page'] = page
        # add offset info
        if offset:
            headers['offset'] = offset

        response = self.call_api(url, HttpMethod.GET, headers=headers)
        
        if self.is_response_json(response):
            return response.json()
        else:
            return response.text

    def get_user(self, user_id):
        
        path = '/users/{}'.format(user_id)
        url = urllib.parse.urljoin(self._api_base, path)
        response = self.call_api(url, HttpMethod.GET)
        if self.is_response_json(response):
            return response.json()
        else:
            return response.text

    def post_user(self):
        # send json data

        path = 'users'
        url = urllib.parse.urljoin(self._api_base, path)
        headers = {'Content-Type': 'application/json'}
        payload = {'some': 'data'}
        data = json.dumps(payload)

        response = self.call_api(url, HttpMethod.POST, headers=headers, data=data)


In [ ]:
# test
unit = SampleAPIHandler()
unit.get_users()

## retry

You have to just substitute session with requests_retry_session.

In [ ]:
def requests_retry_session(session=None,
                           retries=3,
                           backoff_factor=0.3,
                           status_forcelist=(500, 502, 504)):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

## binary

In [ ]:
# content-encoding  gzip ?
# Binary Response
    r.content

output_file_path = ''

# important to have stream=True
r = requests.get(url, stream=True)
if r.status_code == 200:
    with open(output_file_path, 'wb') as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)
            
            
# {
#     'content-encoding': 'gzip',
#     'transfer-encoding': 'chunked',
#     'connection': 'close',
#     'server': 'nginx/1.0.4',
#     'x-runtime': '148ms',
#     'etag': '"e1ca502697e5c9317743dc078f67693f"',
#     'content-type': 'application/json'
# }


## caching 

In [ ]:
# Simple Key Value caching

# using redis, ...



# Http caching
    # etag
    
    # 304 Not Modified
    
    

### HTTP cache


In [ ]:
# install
!pip install CacheControl

In [42]:
# import
from cachecontrol import CacheControl

In [ ]:
def add_cache(session, capacity=100):
    # in-memory dictionary
    session = session or requests.Session()
    return CacheControl(session)


### redis cache

In [43]:
import redis
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.redis_cache import RedisCache


pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
r = redis.Redis(connection_pool=pool)
sess = CacheControl(requests.Session(), RedisCache(r))


In [ ]:
# testing
http://httpstat.us/

In [ ]:
# Http 2 